# **Volcengine Agent Development Kit 教程**

## 介绍

VeADK 是由火山引擎推出的一套面向智能体开发的全流程框架，旨在为开发者提供面向智能体构建、云端部署、评测与优化的全流程开发。

相较于现有的智能体开发框架，VeADK 具备与火山引擎产品体系深度融合的优势，帮助开发者更高效地构建企业级 AI 智能体应用。

## 配置

### 安装 VeADK

In [ ]:
%pip install veadk-python --quiet

> 如果你在本地使用的终端是`zsh`，当你进行`pip install`时，**依赖包名称应当被双引号包裹起来**，否则会解析错误。
>
> 你应当使用：
> ```bash
> pip install "veadk-python"
> pip install "agent-pilot-sdk>=0.0.9"
> ```

### 环境变量

In [ ]:
import os

os.environ["MODEL_AGENT_NAME"] = "doubao-seed-1-6-250615"  # <-- 火山方舟大模型名称

os.environ["MODEL_AGENT_API_KEY"] = ""  # <-- 设置火山方舟的API KEY来访问模型

os.environ["LOGGING_LEVEL"] = "ERROR"  # <-- 调整日志等级

## Agent 构建

### 快速开始

初始化Agent过程中，你可以不传入任何参数，VeADK 会自动读取环境变量。查看[完整的 Agent 参数](https://volcengine.github.io/veadk-python/agent/agent#%E9%80%89%E9%A1%B9)。

In [ ]:
from veadk import Agent

agent = Agent()  # 初始化 Agent

response = await agent.run("你好，我是一名 Agent 开发者。")

print(response)

Hello! It's fantastic to connect with a fellow Agent developer. As someone working in this space, you might be building Agents for specific domains like automation, data processing, or even multi-modal interaction. Are you currently focused on a particular project—maybe refining an existing Agent's capabilities, solving integration challenges (like tool APIs or workflow logic), or exploring new use cases? If you have questions about architecture design, optimizing response efficiency, or best practices for Agent-human collaboration, feel free to share—I’d love to help brainstorm or troubleshoot! 😊


### 使用 Runner

生产环境下的最佳实践：使用`Runner`来运行 Agent，通过`app_name`、`user_id`以及`session_id`以实现多租户隔离。

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent()
short_term_memory = ShortTermMemory()  # 短期记忆代表的是用户单次会话内的对话记录

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

# 使用 runner 中的`run`方法来调用 Agent
response = await runner.run(
    messages="你好，我是一名 Agent 开发者。", session_id=session_id
)

print(response)

你好！很高兴认识你这位 Agent 开发者！开发智能 Agent 确实是一项充满挑战又极具创造力的工作呢。你目前是在开发某个特定领域的 Agent（比如数据处理、自动化任务、智能交互等），还是在构建 Agent 框架或工具呢？如果在开发过程中遇到任何问题——比如数据科学模块的设计、代码实现（Python/JavaScript 等）、文档撰写，或者 Agent 与外部工具的集成等，都可以随时告诉我，我很乐意和你一起探讨解决方案！ 😊


### 使用 Tools

初始化 Agent 时，你可以使用`tools`字段来挂载工具：

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent(tools=[get_city_weather])
short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

response = await runner.run(messages="北京的天气怎么样？", session_id=session_id)

print(response)

北京现在天气晴朗，气温25°C。


你还可以体验**火山引擎提供的强大搜索能力**，例如使用`web_search`和`VeSearch`工具等。

使用`web_search`工具：

In [3]:
import os

# 设置火山引擎 AK 和 SK 来使用 web_search 工具
os.environ["VOLCENGINE_ACCESS_KEY"] = ""
os.environ["VOLCENGINE_SECRET_KEY"] = ""

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.builtin_tools.web_search import web_search

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent(tools=[web_search])
short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

response = await runner.run(messages="北京今天的空气质量", session_id=session_id)

print(response)

根据北京市生态环境监测中心数据，**今天（8月15日）北京空气质量指数（AQI）为40，等级为优**。具体污染物浓度如下：  
- PM2.5：38 μg/m³  
- PM10：38 μg/m³  
- O3：24 μg/m³  
- NO2：7 μg/m³  
- SO2：1 μg/m³  
- CO：0.6 mg/m³  

**建议措施**：各类人群可正常活动。  

此外，未来三天（16-18日）空气质量预计维持优至良等级，首要污染物为臭氧（O3），需注意夏季臭氧污染防护。


尝试使用`VeSearch`工具：

In [5]:
import os

# 设置 vesearch 工具的 ENDPOINT 和 API KEY
os.environ["TOOL_VESEARCH_ENDPOINT"] = ""
os.environ["TOOL_VESEARCH_API_KEY"] = ""

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.builtin_tools.vesearch import vesearch

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent(tools=[vesearch])
short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

response = await runner.run(messages="未来七天北京的天气", session_id=session_id)

print(response)

### 北京未来七天天气预报（2025年8月16日-8月22日）  
- **8月16日（周六）**：雷阵雨，24-31℃，南风1级，湿度82%，紫外线指数4，空气质量良。  
- **8月17日（周日）**：雷阵雨，24-30℃，东风1级，湿度85%，紫外线指数4，空气质量良。  
- **8月18日（周一）**：阴转雷阵雨，24-30℃，南风1级，湿度89%，紫外线指数4，空气质量良。  
- **8月19日（周二）**：雷阵雨，24-29℃，西北风1级，湿度86%，紫外线指数3，空气质量良。  
- **8月20日（周三）**：阴转雷阵雨，24-31℃，东南风1级，湿度86%，紫外线指数4，空气质量良。  
- **8月21日（周四）**：小雨转阴天，24-32℃，西南风1级，湿度80%，紫外线指数3，空气质量信息暂缺。  
- **8月22日（周五）**：晴，22-30℃，南风1级，湿度69%，紫外线指数5，空气质量信息暂缺。  

### 重要提示  
- **预警信息**：15日傍晚至16日早晨有暴雨（小时雨强可达30毫米以上）及雷电，局地伴7级短时大风，山区需防范山洪、泥石流等次生灾害。  
- **生活建议**：未来三天（15-17日）不适宜洗车和户外活动，需随身携带雨具，注意防雷防雨。


### 短期记忆

你可以初始化一个`Local`模式的短期记忆，它只会存在与内存中。其仅在一次调用中会有记忆。

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent()
short_term_memory = ShortTermMemory(
    backend="local"
)  # 指定 local 后端，或直接 ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

response = await runner.run(
    messages=["我叫VeADK", "你还记得我叫什么吗？"], session_id=session_id
)

print(response)

当然记得！你叫VeADK，很高兴再次和你交流 😊


VeADK 还支持你将短期记忆持久化存储在云端，未来的某一时刻你可以加载历史对话。

使用 MySQL 作为短期记忆的数据库后端：

In [ ]:
import os

os.environ["DATABASE_MYSQL_HOST"] = ""
os.environ["DATABASE_MYSQL_USER"] = ""
os.environ["DATABASE_MYSQL_PASSWORD"] = ""
os.environ["DATABASE_MYSQL_DATABASE"] = ""
os.environ["DATABASE_MYSQL_CHARSET"] = "utf8"

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent()
short_term_memory = ShortTermMemory(backend="mysql")  # 指定 mysql 后端
runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

prompt = "我在 7 月 15 日购买了 20 个冰激凌"
response = await runner.run(messages=prompt, session_id=session_id)
print(response)

prompt = "我什么时候买了冰激凌？"
response = await runner.run(messages=prompt, session_id=session_id)
print(response)

原来你在7月15日买了20个冰激凌呀，数量不少呢！是有什么特别的场合需要用到这么多，还是单纯想囤货慢慢吃呀？ 😄 如果你需要记录这个购买信息、做简单的消费统计，或者有其他相关的小需求，随时告诉我，我可以帮你~
You mentioned that you bought the ice cream on July 15th. 😊


### 长期记忆

如果您使用知识库、长期记忆等进阶功能，请进一步安装 veadk-python 中的扩展包：

In [ ]:
%pip install veadk-python[extensions] --quiet

使用 OpenSearch 作为长期记忆的数据库后端：

In [ ]:
import os

# OpenSearch 配置
os.environ["DATABASE_OPENSEARCH_HOST"] = ""
os.environ["DATABASE_OPENSEARCH_PORT"] = ""
os.environ["DATABASE_OPENSEARCH_USERNAME"] = ""
os.environ["DATABASE_OPENSEARCH_PASSWORD"] = ""

# Embedding 配置（使用 OpenSearch 时，需要对文本进行向量化处理）
# 设置访问火山方舟的 Embedding 模型
os.environ["MODEL_EMBEDDING_NAME"] = "doubao-embedding-text-240715"
os.environ["MODEL_EMBEDDING_API_BASE"] = "https://ark.cn-beijing.volces.com/api/v3/"
os.environ["MODEL_EMBEDDING_DIM"] = "2560"
os.environ["MODEL_EMBEDDING_API_KEY"] = ""

In [ ]:
from veadk import Agent, Runner
from veadk.memory.long_term_memory import LongTermMemory
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"

# 初始化一个长期记忆，采用 OpenSearch 向量化存储
# 长期记忆是跨 Session 的
long_term_memory = LongTermMemory(
    backend="opensearch", app_name=app_name, user_id=user_id
)

agent = Agent(long_term_memory=long_term_memory)

runner = Runner(
    agent=agent,
    app_name=app_name,
    user_id=user_id,
    short_term_memory=ShortTermMemory(),
)


# ===== 插入记忆 =====
session_id = "veadk_playground_session"
teaching_prompt = "我上周五购买了一支冰激凌。"

await runner.run(messages=teaching_prompt, session_id=session_id)
await runner.save_session_to_long_term_memory(
    session_id=session_id
)  # 将 teaching prompt 和智能体回答保存到长期记忆中


# ===== 检验记忆 =====
session_id = "veadk_playground_session_2"  # 使用一个新的 Session 来检测跨 Session 检索
student_prompt = "我上周五购买了什么? 用中文回答我。"

response = await runner.run(messages=student_prompt, session_id=session_id)

print(response)

你上周五购买了一支冰激凌。


使用 Viking Memory 作为长期记忆的后端：

In [8]:
import os

# Viking 记忆库配置
os.environ["DATABASE_VIKING_PROJECT"] = ""
os.environ["DATABASE_VIKING_REGION"] = "cn-beijing"

In [ ]:
from veadk import Agent, Runner
from veadk.memory.long_term_memory import LongTermMemory
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"

# 初始化一个长期记忆，采用 Viking Memory 存储
# 长期记忆是跨 Session 的
long_term_memory = LongTermMemory(backend="viking", app_name=app_name, user_id=user_id)

agent = Agent(long_term_memory=long_term_memory)

runner = Runner(
    agent=agent,
    app_name=app_name,
    user_id=user_id,
    short_term_memory=ShortTermMemory(),
)


# ===== 插入记忆 =====
session_id = "veadk_playground_session_3"
teaching_prompt = "我这周三购买了三本书。"

await runner.run(messages=teaching_prompt, session_id=session_id)
await runner.save_session_to_long_term_memory(
    session_id=session_id
)  # 将 teaching prompt 和智能体回答保存到长期记忆中

由于初始使用 Viking Memory 需要构建索引，因此需要等待 1-2 分钟。等待过后，执行如下代码检验：

In [ ]:
# ===== 检验记忆 =====
session_id = "veadk_playground_session_4"  # 使用一个新的 Session 来检测跨 Session 检索
student_prompt = "我这周三购买了什么? 用中文回答我。"

response = await runner.run(messages=student_prompt, session_id=session_id)

print(response)

根据记忆记录，你在2025年8月13日（周三）购买了三本书。


### 知识库

如果您使用知识库、长期记忆等进阶功能，请进一步安装 veadk-python 中的扩展包：

In [ ]:
%pip install veadk-python[extensions] --quiet

使用 OpenSearch 作为知识库的后端：

In [ ]:
import os

# OpenSearch 配置
os.environ["DATABASE_OPENSEARCH_HOST"] = ""
os.environ["DATABASE_OPENSEARCH_PORT"] = "9200"
os.environ["DATABASE_OPENSEARCH_USERNAME"] = ""
os.environ["DATABASE_OPENSEARCH_PASSWORD"] = ""

# 设置访问火山方舟的 Embedding 模型
os.environ["MODEL_EMBEDDING_NAME"] = "doubao-embedding-text-240715"
os.environ["MODEL_EMBEDDING_API_BASE"] = "https://ark.cn-beijing.volces.com/api/v3/"
os.environ["MODEL_EMBEDDING_DIM"] = "2560"
os.environ["MODEL_EMBEDDING_API_KEY"] = ""

In [12]:
# 准备知识库文档并保存到本地
content = """
        The secret of red is red_000000.
        The secret of green is green_000111.
        The secret of blue is blue_000222.
        The secret of yellow is yellow_000333.
"""

knowledgebase_file = "/tmp/knowledgebase.md"
with open(knowledgebase_file, "w", encoding="utf-8") as f:
    f.write(content)

print(f"Knowledgebase file path: {knowledgebase_file}")

Knowledgebase file path: /tmp/knowledgebase.md


In [ ]:
from veadk import Agent, Runner
from veadk.knowledgebase.knowledgebase import KnowledgeBase
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"


knowledgebase = KnowledgeBase(
    backend="opensearch", app_name=app_name
)  # 指定 opensearch 后端
knowledgebase.add_from_files(files=[knowledgebase_file])

agent = Agent(knowledgebase=knowledgebase)

runner = Runner(
    agent=agent,
    short_term_memory=ShortTermMemory(),
    app_name=app_name,
    user_id=user_id,
)

response = await runner.run(
    messages="Tell me the secret of green.", session_id=session_id
)
print(response)

The secret of green is green_000111.


使用 Viking DB 作为知识库的后端：

In [14]:
import os

# Viking DB 配置
os.environ["DATABASE_VIKING_PROJECT"] = ""
os.environ["DATABASE_VIKING_REGION"] = "cn-beijing"

# 存储桶配置
os.environ["DATABASE_TOS_ENDPOINT"] = "tos-cn-beijing.volces.com"
os.environ["DATABASE_TOS_REGION"] = "cn-beijing"
os.environ["DATABASE_TOS_BUCKET"] = ""

In [15]:
import os
import requests

# 准备知识库文档并保存到本地
pdf_url = "https://arxiv.org/pdf/1706.03762"
pdf_path = "/tmp/knowledgebase.pdf"
resp = requests.get(pdf_url)
with open(pdf_path, "wb") as f:
    f.write(resp.content)

print(f"Knowledgebase file path: {pdf_path}")

Knowledgebase file path: /tmp/knowledgebase.pdf


In [ ]:
from veadk import Agent
from veadk.knowledgebase.knowledgebase import KnowledgeBase
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

knowledgebase = KnowledgeBase(backend="viking", app_name=app_name)  # 指定 viking 后端

knowledgebase.add_from_files(files=[pdf_path])  # 直接添加文档，无需手动切片

agent = Agent(knowledgebase=knowledgebase)

由于初始使用 Viking DB 需要构建索引，因此需要等待 3-4 分钟。等待过后，执行如下代码检验：

In [ ]:
from veadk import Runner

runner = Runner(
    agent=agent,
    short_term_memory=ShortTermMemory(),
    app_name=app_name,
    user_id=user_id,
)

response = await runner.run(
    messages="What is `Multi-Head Attention`?", session_id=session_id
)
print(response)

`Multi-Head Attention` is a key component in the Transformer architecture, proposed in the paper *"Attention Is All You Need"* (Vaswani et al., 2017). It extends the standard attention mechanism by allowing the model to **simultaneously learn different types of attention patterns** from multiple "heads" (parallel attention functions), thereby capturing richer contextual relationships in the input data.


### **Core Idea**  
Instead of computing a single attention function over the input, multi-head attention splits the query, key, and value vectors into multiple smaller subspaces (heads), computes scaled dot-product attention independently for each subspace, and then concatenates the results. This enables the model to focus on **different positional or semantic relationships** (e.g., local dependencies, long-range dependencies, syntactic vs. semantic cues) in parallel.


### **Mathematical Formulation**  
Given queries \( Q \), keys \( K \), and values \( V \):  
1. **Linear Projection

### 多 Agent 协同

你可以通过设置`sub_agents`字段来进行多 Agent 之间的本地协同。

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

short_term_memory = ShortTermMemory()

# sub agents
python_coding_agent = Agent(
    name="python_coder",
    description="擅长使用 Python 编程语言来解决问题。",
    instruction="""使用 Python 语言来解决问题。
    注意，你生成的代码第一行需要有注释，标明作者是`python-coder`。""",
)

java_coding_agent = Agent(
    name="java_coder",
    description="擅长使用 Java 编程语言来解决问题。",
    instruction="""使用 Java 语言来解决问题。
    注意，你生成的代码第一行需要有注释，标明作者是`java-coder`。""",
)

# root agent
coding_agent = Agent(
    name="coding_agent",
    description="可以调用适合的智能体来解决用户问题。",
    instruction="调用适合的智能体来解决用户问题。",
    sub_agents=[python_coding_agent, java_coding_agent],
)


runner = Runner(
    agent=coding_agent,
    short_term_memory=ShortTermMemory(),
    app_name=app_name,
    user_id=user_id,
)

response = await runner.run(
    messages="使用 Python 帮我写一段快速排序的代码。", session_id=session_id
)

print(response)

```python
# 作者：python-coder
def quick_sort(arr):
    """
    使用快速排序算法对列表进行排序
    
    参数:
        arr: 需要排序的列表
        
    返回:
        排序后的列表
    """
    if len(arr) <= 1:
        return arr
    
    # 选择基准元素（这里选择中间元素）
    pivot = arr[len(arr) // 2]
    
    # 分区：小于基准、等于基准、大于基准
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    
    # 递归排序并合并结果
    return quick_sort(left) + middle + quick_sort(right)

# 测试示例
if __name__ == "__main__":
    test_arr = [3, 6, 8, 10, 1, 2, 1]
    print("排序前:", test_arr)
    sorted_arr = quick_sort(test_arr)
    print("排序后:", sorted_arr)
```

这段代码实现了快速排序算法，具有以下特点：
1. 采用递归实现，代码简洁易懂
2. 选择中间元素作为基准，避免了极端情况下的性能退化
3. 使用列表推导式进行分区操作，代码更简洁
4. 处理了重复元素的情况（通过单独的middle列表）
5. 包含完整的文档字符串和测试示例

算法时间复杂度平均为O(n log n)，最坏情况下为O(n²)，空间复杂度为O(n)。如果需要原地排序版本（空间复杂度O(log n)），可以实现更复杂的指针交换分区方式。


## 可观测性

### 本地观测

可以通过初始化 OpentelemetryTracer 来将 Agent 的调用栈存储为本地文件：

In [ ]:
import json

from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather
from veadk.tracing.telemetry.opentelemetry_tracer import OpentelemetryTracer

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

tracer = OpentelemetryTracer()

agent = Agent(
    tools=[get_city_weather], tracers=[tracer]
)  # 创建一个配置 tracers 的 Agent

short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

# 如果在 agent 中设置了tracer，
# 并且在 `run` 中指定保存，runner将会尝试保存 tracing 文件
await runner.run(
    messages="北京的天气怎么样？", session_id=session_id, save_tracing_data=True
)

print(f"Tracing file path: {tracer._trace_file_path}")

with open(tracer._trace_file_path, "r") as f:
    tracing_content = f.read()

print(f"Tracing file content:\n{json.loads(tracing_content)}")

Tracing file path: /tmp/veadk_global_tracer_veadk-playground-user_veadk-playground-session_b204302b2d0e24b6e26ca914a5fef478.json
Tracing file content:
[{'name': 'execute_tool get_city_weather', 'span_id': 179319007772593823, 'trace_id': 236624329413565051863190183785302783096, 'start_time': 1755172431614674977, 'end_time': 1755172431617289457, 'attributes': {'session.id': 'veadk-playground-session', 'user.id': 'veadk-playground-user', 'agent.name': 'veAgent', 'app.name': 'veadk-playground-app', 'gen_ai.system': 'gcp.vertex.agent', 'gen_ai.operation.name': 'execute_tool', 'gen_ai.tool.name': 'get_city_weather', 'gen_ai.tool.description': 'Retrieves the weather information of a given city. the args must in English', 'gen_ai.tool.call.id': 'call_2e7gig0mp86rzc1ffwf5ftdc', 'gcp.vertex.agent.tool_call_args': '{"city": "Beijing"}', 'gcp.vertex.agent.event_id': '04616e67-6881-43d0-8200-09c7c069be06', 'gcp.vertex.agent.tool_response': '{"result": "Sunny, 25°C"}', 'gcp.vertex.agent.llm_request'

### 云端观测

需要设置上报器（exporter），VeADK 当前支持将 Tracing 数据上报至火山引擎 TLS、火山引擎 APMPlus 以及火山引擎 Coze Loop 平台。它们分别对应不同的上报器。

初始化上报器之前需要进行环境变量设置：

In [ ]:
import os

# TLS 上报器环境变量
os.environ["OBSERVABILITY_OPENTELEMETRY_TLS_ENDPOINT"] = (
    "https://tls-cn-beijing.volces.com:4318/v1/traces"
)
os.environ["OBSERVABILITY_OPENTELEMETRY_TLS_REGION"] = "cn-beijing"
os.environ["OBSERVABILITY_OPENTELEMETRY_TLS_SERVICE_NAME"] = ""

# APMPlus 上报器环境变量
os.environ["OBSERVABILITY_OPENTELEMETRY_APMPLUS_ENDPOINT"] = (
    "http://apmplus-cn-beijing.volces.com:4317"
)
os.environ["OBSERVABILITY_OPENTELEMETRY_APMPLUS_SERVICE_NAME"] = ""
os.environ["OBSERVABILITY_OPENTELEMETRY_APMPLUS_API_KEY"] = ""

# Coze Loop 上报器环境变量
os.environ["OBSERVABILITY_OPENTELEMETRY_COZELOOP_ENDPOINT"] = (
    "https://api.coze.cn/v1/loop/opentelemetry/v1/traces"
)
os.environ["OBSERVABILITY_OPENTELEMETRY_COZELOOP_SERVICE_NAME"] = ""
os.environ["OBSERVABILITY_OPENTELEMETRY_COZELOOP_API_KEY"] = ""

初始化上报器，并装配到 Agent 中：

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather
from veadk.tracing.telemetry.exporters.apmplus_exporter import APMPlusExporter
from veadk.tracing.telemetry.exporters.cozeloop_exporter import CozeloopExporter
from veadk.tracing.telemetry.exporters.tls_exporter import TLSExporter
from veadk.tracing.telemetry.opentelemetry_tracer import OpentelemetryTracer

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

exporters = [
    CozeloopExporter(),
    APMPlusExporter(),
    TLSExporter(),
]  # 初始化 tracing 上报器
tracer = OpentelemetryTracer(exporters=exporters)

agent = Agent(
    tools=[get_city_weather], tracers=[tracer]
)  # 创建一个配置 tracers 的 Agent

short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

await runner.run(messages="北京的天气怎么样？", session_id=session_id)

print(f"Tracing file path: {tracer._trace_file_path}")

Tracing file path: /tmp/veadk_global_tracer_veadk_playground_user_veadk_playground_session_2a70673f5f428cab82e0c68266a19560.json


最后，在云端平台可查看 Tracing 数据：

*   Coze Loop 平台：https://www.coze.cn/loop
    ![Coze loop 监控图](https://github.com/volcengine/veadk-python/blob/main/assets/images/tutorial-observation-cozeloop.png?raw=true)

*   APMPlus 平台：https://www.volcengine.com/product/apmplus
    ![APMPlus 监控图](https://github.com/volcengine/veadk-python/blob/main/assets/images/tutorial-observation-apmplus-tracing.png?raw=true)

*   TLS 平台：https://www.volcengine.com/product/tls




## 评测

### 运行时数据转换

可以通过 runtime recorder 将运行时的数据直接保存为 Google ADK 兼容的 Evaluation Set 文件（json 格式）：

In [ ]:
import json

from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather

app_name = "veadk_playground_app"
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

agent = Agent(tools=[get_city_weather])
short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent, short_term_memory=short_term_memory, app_name=app_name, user_id=user_id
)

await runner.run(messages="北京的天气怎么样？", session_id=session_id)

# 调用 runner 中的`save_eval_set`来保存评测集文件到本地
eval_set_path = await runner.save_eval_set(session_id=session_id)

print(f"Evaluation file path: {eval_set_path}")

with open(eval_set_path, "r") as f:
    data = json.load(f)
print(f"Evaluation file content\n{data}")

Evaluation file path: /tmp/veadk-playground-app/default.evalset.json
Evaluation file content
{'eval_set_id': 'default', 'name': 'default', 'description': None, 'eval_cases': [{'eval_id': 'veadk_eval_20250814115717', 'conversation': [{'invocation_id': 'e-55f9f9f1-0cd0-45d4-82e9-8d4805215e19', 'user_content': {'parts': [{'video_metadata': None, 'thought': None, 'inline_data': None, 'file_data': None, 'thought_signature': None, 'code_execution_result': None, 'executable_code': None, 'function_call': None, 'function_response': None, 'text': '北京的天气怎么样？'}], 'role': 'user'}, 'final_response': {'parts': [{'video_metadata': None, 'thought': None, 'inline_data': None, 'file_data': None, 'thought_signature': None, 'code_execution_result': None, 'executable_code': None, 'function_call': None, 'function_response': None, 'text': '北京当前天气晴朗，气温25°C。'}], 'role': None}, 'intermediate_data': {'tool_uses': [{'id': 'call_6wbw4k3urxyd4hy8cil83n9m', 'args': {'city': 'Beijing'}, 'name': 'get_city_weather'}], '

### 使用评测器进行评测

在使用评测之前，需要安装评测相关的依赖：

In [ ]:
%pip install veadk-python[eval] --quiet

评判模型的环境变量设置：

In [ ]:
import os

os.environ["MODEL_JUDGE_API_BASE"] = "https://ark.cn-beijing.volces.com/api/v3/"
os.environ["MODEL_JUDGE_NAME"] = "doubao-seed-1-6-250615"
os.environ["MODEL_JUDGE_API_KEY"] = ""

以 Deepval 为例：

In [ ]:
from deepeval.metrics import GEval, ToolCorrectnessMetric
from deepeval.test_case import LLMTestCaseParams

from veadk import Agent
from veadk.tools.demo_tools import get_city_weather
from veadk.config import getenv
from veadk.evaluation.deepeval_evaluator import DeepevalEvaluator
from veadk.prompts.prompt_evaluator import eval_principle_prompt

agent = Agent(tools=[get_city_weather])

evaluator = DeepevalEvaluator(
    agent=agent, judge_model_api_key=getenv("MODEL_JUDGE_API_KEY")
)

judge_model = (
    evaluator.judge_model
)  # 如果你希望使用方舟作为 judge model，那么 evaluator 会帮你自动初始化

metrics = [
    GEval(
        threshold=0.8,
        name="Base Evaluation",
        criteria=eval_principle_prompt,
        evaluation_params=[
            LLMTestCaseParams.INPUT,
            LLMTestCaseParams.ACTUAL_OUTPUT,
            LLMTestCaseParams.EXPECTED_OUTPUT,
        ],
        model=judge_model,  # 这里需要传入 judge model
    ),
    ToolCorrectnessMetric(threshold=0.5),
]

await evaluator.evaluate(eval_set_file_path=eval_set_path, metrics=metrics)

✨ You're running DeepEval's latest Base Evaluation [GEval] Metric! (using None (Local Model), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Tool Correctness Metric! (using None, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Base Evaluation [GEval] (score: 1.0, threshold: 0.8, strict: False, evaluation model: None (Local Model), reason: All evaluation steps are satisfied. The Actual Output ('北京当前天气晴朗，气温为25°C。'), Input ('北京的天气怎么样？'), and Expected Output ('北京当前天气晴朗，气温25°C。') are all human-readable (coherent, no garbled text). Both Actual and Expected Outputs are relevant to the Input, which asks about Beijing's weather., error: None)
  - ✅ Tool Correctness (score: 1.0, threshold: 0.5, strict: False, evaluation model: None, reason: All expected tools ['get_city_weather'] were called (order not considered)., error: None)

For test case:

  - input: 北京的天气怎么样？
  - actual output: 北京当前天气晴朗，气温为25°C。
  - expected output: 北京当前天气晴朗，气温25°C。
  - context: ['actual_conversation_history: Empty', 'expect_conversation_history: Empty']
  - retrieval context: None


Overall Metric Pass Rates

Base Evaluation [GEval]: 100.00% pass rate
Tool Correctness: 100.00% pass rate




✓ Evaluation completed 🎉! (time taken: 24.09s | token cost: 0.0 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Base Evaluation [GEval]', threshold=0.8, success=True, score=1.0, reason="All evaluation steps are satisfied. The Actual Output ('北京当前天气晴朗，气温为25°C。'), Input ('北京的天气怎么样？'), and Expected Output ('北京当前天气晴朗，气温25°C。') are all human-readable (coherent, no garbled text). Both Actual and Expected Outputs are relevant to the Input, which asks about Beijing's weather.", strict_mode=False, evaluation_model='None (Local Model)', error=None, evaluation_cost=0.0, verbose_logs='Criteria:\n\nYou are a LLM for evaluating other models\' responses. Note:\n- The response maybe generated by some uncertainty tools (e.g., online-search, random number), you just need to consider whether the response is human-readable, rather than focus on the specific content. Because the specific content maybe different at different time.\n \n \nEvaluation Steps:\n[\n    "Check if the Actual Output is human-readable (co

## Prompt 优化

使用火山引擎 PromptPilot 来进行 Agent 的系统提示词（System Prompt）优化。

In [ ]:
# 安装火山引擎提供的依赖
%pip install agent-pilot-sdk>=0.1.2

您可以从 Prompt Pilot 产品[官方页面](https://promptpilot.volcengine.com/)获取 KEY 和 Workspace ID，在下方设置后访问服务：

In [ ]:
import os

os.environ["PROMPT_PILOT_API_KEY"] = ""
os.environ["PROMPT_PILOT_WORKSPACE_ID"] = ""

定义一个简单的 Agent，并且让它携带天气查询的工具，体验 Prompt 优化前后的差异。

In [ ]:
from veadk import Agent
from veadk.tools.demo_tools import get_city_weather

agent = Agent(instruction="你是一个智能体", tools=[get_city_weather])

使用 Prompt Pilot 进行优化：

In [ ]:
from veadk.integrations.ve_prompt_pilot.ve_prompt_pilot import VePromptPilot

prompt_pilot = VePromptPilot(
    api_key=os.getenv("PROMPT_PILOT_API_KEY"),
    workspace_id=os.getenv("PROMPT_PILOT_WORKSPACE_ID"),
)

refined_prompt = prompt_pilot.optimize(agents=[agent])

Optimized prompt for agent veAgent:
# Role
你是veAgent，是由VeADK团队开发的AI智能体，专长于数据科学、文档编写和软件开发领域。

# Task Requirements
## 基于你的角色和对话历史，合理推断并延续对话。需考虑以下方面：
- **对话风格**：语言表达应专业、准确、清晰，符合技术领域的交流风格，避免模糊或歧义的表述。
- **沟通目的**：围绕用户在数据科学、文档编写和软件开发方面的问题展开对话，提供专业的建议、解决方案或相关信息。不偏离用户的问题核心，确保对话的连贯性和有效性。
- **专业能力展现**：在涉及数据科学、文档编写和软件开发的专业知识时，提供准确、详细且有价值的内容。若用户需求涉及特定工具或技术，可结合自身专长进行解答。

## 额外要求
- **简洁高效**：回答应简洁明了，避免冗长复杂的句子和不必要的修饰，确保信息能够高效传达。
- **专业态度**：始终保持专业的态度，不随意发表没有依据的观点。对于不熟悉的领域，坦诚告知用户，并提供可能的解决途径。 


## 云端部署

### 部署到火山引擎 FaaS 平台

详见[部署上云](https://volcengine.github.io/veadk-python/deploy.html)。

### 端云协同

VeADK 提供了在本地直接调用/操作部署在火山引擎 FaaS 平台的 Agent 的便捷接口，方便你与云端 Agent 协作。

**以 A2A 方式调用部署的 Agent**

In [ ]:
# 部署在 VeFaaS 上的应用名称（Application name）
vefaas_application_name = ""

In [ ]:
from veadk.cloud.cloud_app import CloudApp

# 设置会话属性
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

cloud_app = CloudApp(vefaas_application_name=vefaas_application_name)

print(f"VeFaaS 应用 ID: {cloud_app.vefaas_application_id}")

# 发送消息
response_message = await cloud_app.message_send(
    message="你好，我是一名 Agent 开发者。", user_id=user_id, session_id=session_id
)

print(f"返回的消息源: {cloud_app.vefaas_endpoint}")
print(f"返回的消息内容: {response_message.parts[0].root.text}")

**以 MCP 方式调用部署的Agent**

使用此方式需安装 `fastmcp` 库：

In [ ]:
%pip install fastmcp --quiet

In [ ]:
# 部署在 VeFaaS 上的应用名称（Application name）
vefaas_application_name = ""

In [ ]:
from veadk.cloud.cloud_app import CloudApp
from fastmcp.client import Client

# 设置会话属性
user_id = "veadk_playground_user"
session_id = "veadk_playground_session"

cloud_app = CloudApp(vefaas_application_name=vefaas_application_name)

endpoint = cloud_app._get_vefaas_endpoint()
print(f"VeFaaS 应用 Endpoint: {endpoint}")

client = Client(f"{endpoint}/mcp")

async with client:
    # 列出所有工具
    tools = await client.list_tools()
    print(f"云部署的 MCP 工具: {tools}")

    # 通过 MCP 的方式调用云端 Agent
    res = await client.call_tool(
        "run_agent",
        {
            "user_input": "你好，我是一名 Agent 开发者。",
            "session_id": session_id,
            "user_id": user_id,
        },
    )
    print(f"返回的消息内容: {res}")